In [1]:
import os
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import re
from nltk.tokenize import word_tokenize
import sklearn

## Load whole dataset to work with DA patterns

In [2]:
df = pd.read_csv('./data_TM2/synt_annotated_data.csv', index_col=0)

## Repair cases

Next round to detecting misunderstanding repair happens

##### forma mais facil: em uma ventana fixa de tamanho 5, rephrasing é se o cosine vector for acima de 0.7 (ou algum outro numero a ser definido)

##### de momento de forma naive, sem interessar quem é o speaker ou se a ventana está dentro do mesmo diálogo ou nao


In [3]:
# #ideas: make column with number 1 if 2 before and 2 after repair initiator
# #make another dataframe which is a subset with 2 previous and 2 posterior values
# # df.loc[df['DA_rep_init']=='repair_initiator']:

# df.loc[df['DA_rep_init']=='repair_initiator','repair_window'] = df['new_text']
# df.loc[df['DA_rep_init'].shift(1)=='repair_initiator','repair_window'] = df['new_text'].shift(1)
# df.loc[df['DA_rep_init'].shift(2)=='repair_initiator','repair_window'] = df['new_text'].shift(2)
# df.loc[df['DA_rep_init'].shift(-1)=='repair_initiator','repair_window'] = df['new_text'].shift(-1)
# df.loc[df['DA_rep_init'].shift(-2)=='repair_initiator','repair_window'] = df['new_text'].shift(-2)
# df.loc[df['DA_rep_init']=='repair_initiator']
# df.iloc[310:320,:]
# # df

## Repair second case from before

In [4]:
# #THIS WORKS! need to apply all filters
# # df.loc[df['column name'] condition, 'new column name'] = 'value if condition is met'

# df.loc[(df['speaker'] != df['speaker'].shift(1)) & (df['DA_rep_init'].shift(1) == 'repair_initiator'), 'teste_repair_case'] = 'SECOND'

# df['teste_repair_case'].value_counts()

In [5]:
# df.loc[df['teste_repair_case'] == 'SECOND']
# #second is more an ANSWER (i.e. next round) to: sorry, what did you say? OR what did you mean?

In [6]:
# print('a case of self repair: ')
# print(df.iloc[1708,5:9])
# print('\n\n')
# print(df.iloc[1709,4:6])

## Cosine similarity for paraphrasing
## Word2vec

In [7]:
# from gensim.test.utils import common_texts
# from gensim.models import Word2Vec

# model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
# model.save("word2vec.model")

In [8]:
# model = Word2Vec.load("word2vec.model")
# model.train([["hello", "world"]], total_examples=1, epochs=1)

In [9]:
# vector = model.wv['computer']  # get numpy vector of a word
# sims = model.wv.most_similar('computer', topn=10)  # get other similar words

In [10]:
# from gensim.models import KeyedVectors

# # Store just the words + their trained embeddings.
# word_vectors = model.wv
# word_vectors.save("word2vec.wordvectors")

# # Load back with memory-mapping = read-only, shared across processes.
# wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

# vector = wv['computer']  # Get numpy vector of a word

In [11]:
# from gensim.test.utils import datapath

# # Load a word2vec model stored in the C *text* format.
# wv_from_text = KeyedVectors.load_word2vec_format(datapath('word2vec_pre_kv_c'), binary=False)
# # Load a word2vec model stored in the C *binary* format.
# wv_from_bin = KeyedVectors.load_word2vec_format(datapath("euclidean_vectors.bin"), binary=True)


In [12]:
# # Pretrained models
# # Gensim comes with several already pre-trained models, in the Gensim-data repository:

# import gensim.downloader
# # Show all available models in gensim-data
# print(list(gensim.downloader.info()['models'].keys()))

In [13]:
# # Download the "glove-twitter-25" embeddings
# glove_vectors = gensim.downloader.load('word2vec-google-news-300')

# # Use the downloaded vectors as usual:
# glove_vectors.most_similar('twitter')

## Pre-trained word2vec with cosine distance

References: 
- https://code.google.com/archive/p/word2vec/
- https://huggingface.co/fse/word2vec-google-news-300
- https://radimrehurek.com/gensim/models/word2vec.html <<<<<<<

In [14]:
# # Load word embedding model
# import gensim

# word2vec = './GoogleNews-vectors-negative300.bin.gz'
# language_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec, binary=True)

In [15]:
# #Word Cosine distance

# language_model.most_similar('hello')

In [16]:
# from nltk import RegexpTokenizer
# from nltk.tokenize import word_tokenize

# sentence  = "Hi, there. How are you I doing?"

# tokenizer = RegexpTokenizer(r"\w+") #remove punctuation
# new_words = tokenizer.tokenize(sentence, )

# [print(word +'\n' + str(language_model.most_similar(word))) for word in new_words]

# #problem1: just single words
# #problem2: gives erros when it doesn't co ntain word: example 'and'



## Tutorial doc2vec > for sentences
https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py

In [17]:
# from gensim.test.utils import common_texts
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
# model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [18]:
# from gensim.test.utils import get_tmpfile

# fname = get_tmpfile("my_doc2vec_model")

# model.save(fname)
# model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [19]:
# #vector of tokenized words

# vector = model.infer_vector(["system", "response"])
# vector

In [20]:
# documents

# doc2vec working
https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py

training our model using the Lee Background Corpus included in gensim. This corpus contains 314 documents selected from the Australian Broadcasting

In [21]:
# # Set file names for train and test data
# test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
# lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
# lee_test_file = os.path.join(test_data_dir, 'lee.cor')

# df['tokenized_text'] = [word_tokenize(da) for da in df['new_text']]

In [22]:
import os
import gensim
from sklearn.model_selection import train_test_split
import smart_open


train, test = train_test_split(df,train_size=393180, shuffle=False) #split taking into account full dialogue. Not splitting in middle of dialogue
train, val = train_test_split(train,train_size=0.1, shuffle=False) #só pra testar coisas 0.001 funciona bem! 0.01 20s 

Below, we define a function to:

- open the train/test file (with latin encoding)

- read the file line-by-line

- pre-process each line (tokenize text into individual words, remove punctuation, set to lowercase, etc)

The file we’re reading is a corpus. Each line of the file is a document.

In [23]:
def read_corpus(fname, tokens_only=False):
#     with smart_open.open(fname, encoding="iso-8859-1") as f: #### FOR MY CODE COMMENT THIS LINE #####
        for i, line in enumerate(fname): #### FOR MY CODE PUT (fname) ##### default is (f)
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

# train_corpus = list(read_corpus(lee_train_file))
# test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

train_corpus = list(read_corpus(train['new_text']))
test_corpus = list(read_corpus(test['new_text'], tokens_only=True))

In [24]:
#training corpus one instance example
print(test_corpus[:2])
len(test_corpus)

[['what', 'is', 'the', 'latest', 'headlines', 'related', 'to', 'the', 'philadelphia', 'phillies'], ['series', 'preview', 'philadelphia', 'phillies', 'at', 'milwaukee', 'brewers', 'mlb', 'rumors', 'latest', 'sign', 'phillies', 'will', 'make', 'full', 'court', 'press', 'for', 'manny', 'machado']]


43703

Training the Model

Now, we’ll instantiate a Doc2Vec model with a vector size with 50 dimensions and iterating over the training corpus 40 times. We set the minimum word count to 2 in order to discard words with very few occurrences. (Without a variety of representative examples, retaining such infrequent words can often make a model worse!) Typical iteration counts in the published Paragraph Vector paper results, using 10s-of-thousands to millions of docs, are 10-20. More iterations take more time and eventually reach a point of diminishing returns.

However, this is a very very small dataset (300 documents) with shortish documents (a few hundred words). Adding training passes can sometimes help with such small datasets.

In [25]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=80)

#Build a vocabulary
model.build_vocab(train_corpus)

Essentially, the vocabulary is a list (accessible via model.wv.index_to_key) of all of the unique words extracted from the training corpus. 

Additional attributes for each word are available using the model.wv.get_vecattr() method 

For example, to see how many times penalty appeared in the training corpus:

In [26]:
print(f"Word 'hello' appeared {model.wv.get_vecattr('hello', 'count')} times in the training corpus.")

model.wv.get_vecattr('hello', 'count')

Word 'hello' appeared 625 times in the training corpus.


625

Next, train the model on the corpus. If optimized Gensim (with BLAS library) is being used, this should take no more than 3 seconds. If the BLAS library is not being used, this should take no more than 2 minutes, so use optimized Gensim with BLAS if you value your time.

In [27]:
%%time
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 2min, sys: 26.2 s, total: 2min 26s
Wall time: 1min 59s


In [28]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("my_doc2vec_model")

model.save(fname)
model = gensim.models.doc2vec.Doc2Vec.load(fname)

## Clean code for Window part:

In [342]:
df['unique'] = [n for n in range(len(df))]

In [417]:
from sklearn.metrics import pairwise


def paraphrasing(conversation):
    sentences_window = {}
    nrows = len(conversation['new_text'])
    sentences = conversation['new_text']
    rep_initiators = np.array(conversation['DA_rep_init'])
    window_width = 2
    dialog_id = conversation['conversation_id'].unique()[0]

    for k,sentence,rep_init in zip(range(nrows),sentences,rep_initiators):
        if rep_init== "repair_initiator":
            sentences_window[sentence] = sentences[k-window_width:k+window_width+1]


    rep=pd.DataFrame(sentences_window)#.transpose()
    repair_tok = list(read_corpus(rep.iloc[:,0], tokens_only=True))

    inf_vecs = []
    [inf_vecs.append(model.infer_vector(ut)) for ut in repair_tok]  


    threshold = 0.7
    similarity = []
    paraphrase = []
#     paraph = {}

    for e in range(len(inf_vecs)):
        counter = e+1
        while counter < len(inf_vecs):
            pair_sim = pairwise.cosine_similarity(inf_vecs[e].reshape(1, -1), inf_vecs[counter].reshape(1, -1))
            if pair_sim >= threshold:
                similarity.append((e, rep.iloc[e,0],counter, rep.iloc[counter,0],pair_sim)) #((e,counter, pair_sim))
                paraphrase.append(rep.iloc[counter,0])
            counter +=1
#     paraph[dialog_id] = paraphrasing

    return similarity #paraphrase 
    #P.S: paraphrase is the second sentence said in a sequence of time
    #dict in which key in dialogue number and values are tuples that passes threshold. 
    #Each tuple contains: index_sent_1, sent1, index_sent2, sent2, cosine_similarity


In [418]:
#works! select dialogue ids with repair initiators
dial_w_rep = []
for row in range(len(df)):
    if df['DA_rep_init'][row] == 'repair_initiator':
        dial_w_rep.append(df['conversation_id'][row])
dial_w_rep = list(set(dial_w_rep))

#call function for all dialogues with repair initiator. save in dict in which key=dialogue id and value=paraphrase
paraphr = {}
count =0
for dialogue in dial_w_rep:
    try:
        conversation = df.loc[df['conversation_id'] == dialogue]
        paraphr[dialogue] = paraphrasing(conversation)
    except Exception:
        count+=1
        pass
    
print(count) #number of times it could not compute function
print(len(dial_w_rep))


219
1813


In [419]:
paraphr

{'dlg-c36bfbe7-d2e5-43dd-9e29-72361ea84d3b': [],
 'dlg-8fe9ee31-81df-4047-9085-fac75fa855e5': [(0,
   'Anything else?',
   4,
   'Okay, anything else?',
   array([[0.83314306]], dtype=float32))],
 'dlg-a0d0b8e9-6b6f-4b8e-ba2c-35f9f3b34431': [],
 'dlg-cdcac470-d611-4271-a7d8-5e4aac24ebcf': [],
 'dlg-94131566-6a25-454d-9226-e650fc48d76c': [(1,
   'I would also like an in-flight movie.',
   3,
   'I would also like an in-flight movie.',
   array([[0.9785803]], dtype=float32))],
 'dlg-61942ea5-bab4-4648-ae48-989fe527235f': [],
 'dlg-da5c5d2e-7abc-4ac0-bb47-ff0f5bd7401e': [(0,
   'Let me get one lamb biryani and one sweet kabab.',
   1,
   'One sweet kebab.',
   array([[0.8900376]], dtype=float32)),
  (0,
   'Let me get one lamb biryani and one sweet kabab.',
   3,
   'One lamb biryani, okay.',
   array([[0.7886195]], dtype=float32)),
  (0,
   'Let me get one lamb biryani and one sweet kabab.',
   4,
   'Let me get one sweet kebab.',
   array([[0.911509]], dtype=float32)),
  (1,
   'One swe

In [413]:
#create column in dataframe with rephrase
DA_paraphrase = []

# for key, value in paraphr.items():
#     if len(value) != 0:
# #         print(type(value[0]))
# #         print(key, value, len(value))
#         for v in value:
# #             print(key, v)
# #             df['DA_paraphrase'] = np.where(((df['conversation_id'] == key) & (df['new_text'] == v)), 'Paraphrase', '')                                     
#             for row in range(len(df)):
#                 if df['conversation_id'][row] == key & df['new_text'][row] == v:
#                     DA_paraphrase.append(1)
#                 else:
#                     DA_paraphrase.append(0)

for row in range(len(df)):
#     print(row)
    for key, value in paraphr.items():
#         print(key, value)
        if len(value) != 0:
#             for v in value:
            if (df['conversation_id'][row] == key) & (df['new_text'][row] == max(value)):
                DA_paraphrase.append(1)
        else:
            DA_paraphrase.append(0)

                                                
#     DA_paraphrase.append(np.where(((df['conversation_id'] == key) & (df['new_text'] == v)), 1, 0)                                     

df['DA_paraphrase'] = DA_paraphrase


ValueError: Length of values (486251570) does not match length of index (436883)

In [410]:
# df['DA_paraphrase'].value_counts()
# paraphr
# range(len(df))
(df['conversation_id'][0] == 'dlg-00100680-00e0-40fe-8321-6d81b21bfc4f') & (df['new_text'][0] == 'Hello.')

len(DA_paraphrase)


246402012

In [261]:
# from sklearn.metrics import pairwise

# conversation1 = df.loc[df['conversation_id'] == 'dlg-00e32998-0b0f-47f1-a4f0-2ce90f1718d0']

# sentences_window = {}
# nrows = len(conversation1['new_text'])
# sentences = conversation1['new_text']
# rep_initiators = np.array(conversation1['DA_rep_init'])
# window_width = 2

# for k,sentence,rep_init in zip(range(nrows),sentences,rep_initiators):
#     if rep_init== "repair_initiator":
#         sentences_window[sentence] = sentences[k-window_width:k+window_width+1]

# rep=pd.DataFrame(sentences_window)#.transpose()
# repair_tok = list(read_corpus(rep.iloc[:,0], tokens_only=True))

# inf_vecs = []
# [inf_vecs.append(model.infer_vector(ut)) for ut in repair_tok]  

# threshold = 0.2
# paraphrasing = []

# for e in range(len(inf_vecs)):
#     counter = e+1
#     while counter < len(inf_vecs):
#         pair_sim = pairwise.cosine_similarity(inf_vecs[e].reshape(1, -1), inf_vecs[counter].reshape(1, -1))
#         print(e, counter, pair_sim)
#         if pair_sim >= threshold:
#             print('YES', str(e), str(counter), str(pair_sim))
#             paraphrasing.append((e,counter,pair_sim))
#         counter +=1

# paraphrasing

## !!!!!!!!!!!!!!!!!!!!!!!!!!!!! DON'T run here onwards
Now, we can use the trained model to infer a vector for any piece of text by passing a list of words to the model.infer_vector function. This vector can then be compared with other vectors via cosine similarity.

#### obs: Note that infer_vector() does not take a string, but rather a list of string tokens, which should have already been tokenized the same way as the words property of original training document objects.

In [ ]:
vector = model.infer_vector(['can','you','help','me','finding','a','good','movie','to','watch'])
print(vector)

To assess our new model, we’ll first infer new vectors for each document of the training corpus, compare the inferred vectors with the training corpus, and then returning the rank of the document based on self-similarity. Basically, we’re pretending as if the training corpus is some new unseen data and then seeing how they compare with the trained model. The expectation is that we’ve likely overfit our model (i.e., all of the ranks will be less than 2) and so we should be able to find similar documents very easily. Additionally, we’ll keep track of the second ranks for a comparison of less similar documents.

In [ ]:
%%time
ranks = []
# second_ranks = []

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv)) #sims[0] is most similar
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

#     second_ranks.append(sims[1])

#https://tedboy.github.io/nlps/generated/generated/gensim.models.Doc2Vec.most_similar.html 
#This method computes cosine similarity between a simple mean of the projection weight vectors of the given words and the vectors for each word in the model. 

In [ ]:
train_corpus

In [ ]:
# Let’s count how each document ranks with respect to the training corpus
# NB. Results vary between runs due to random seeding and very small corpus

import collections

counter = collections.Counter(ranks)
print(counter)

Basically, greater than 95% of the inferred documents are found to be most similar to itself and about 5% of the time it is mistakenly most similar to another document. Checking the inferred-vector against a training-vector is a sort of ‘sanity check’ as to whether the model is behaving in a usefully consistent manner, though not a real ‘accuracy’ value.

In [ ]:
#example that indeed it ranks the same sentence together

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'\n SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print( (label, sims[index]))#, ' '.join(train_corpus[sims[index][0]].words)))

Notice above that the most similar document (usually the same text) is has a similarity score approaching 1.0. However, the similarity score for the second-ranked documents should be significantly lower (assuming the documents are in fact different) and the reasoning becomes obvious when we examine the text itself.

In [ ]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
vector

# Testing the model

In [ ]:
import random

# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv)) #This method computes cosine similarity between a simple mean of the projection weight vectors of the given words and the vectors for each word in the model. 

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

In [ ]:
import random
teste = ['hi','how','are','you','doing']

# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(teste)#(test_corpus[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(teste)))#(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

## Window (messy code)

In [32]:
########### acho que posso substituir o convarsation 1 na hora de generalizar por: ########

#####funciona pero super expensive e inecesario. 
#conv_id = df['conversation_id'].unique()

# sub_dfs = [] #lista de slices of dataframes for each conv

# conv_id = df['conversation_id'].unique()
# for unique in conv_id:
#     sub_dfs.append(df.loc[df['conversation_id'] == unique])

#add extra loop simplesmente na hora de criar variaveis selecionando por unique dialogue id

####################  new part being tested ####################  
conv_id = df['conversation_id'].unique()

for unique in conv_id:
    sentences_window = {}
    nrows = len(df['new_text'].loc[df['conversation_id'] == unique])
    sentences = df['new_text'].loc[df['conversation_id'] == unique]
    rep_initiators = np.array(df['DA_rep_init'].loc[df['conversation_id'] == unique])
    window_width = 2
    repair = []

    for k,sentence,rep_init in zip(range(nrows),sentences,rep_initiators):
        if rep_init== "repair_initiator":
            sentences_window[sentence] = sentences[k-window_width:k+window_width+1]
    #         repair.append(list(sentences[k-window_width:k+window_width+1]))
    if sentences_window:
    # rep=pd.DataFrame(repair).transpose()
        rep=pd.DataFrame(sentences_window)#.transpose()
        repair_tok = list(read_corpus(rep.iloc[:,0], tokens_only=True))



TypeError: decoding to str: need a bytes-like object, float found

In [ ]:
#################################################################### 
#OFICIAL HASTA AHORA:

# conversation1 = df.loc[df['conversation_id'] == 'dlg-00e32998-0b0f-47f1-a4f0-2ce90f1718d0']

# sentences_window = {}
# nrows = len(conversation1['new_text'])
# sentences = conversation1['new_text']
# rep_initiators = np.array(conversation1['DA_rep_init'])
# window_width = 2
# # repair = []

# for k,sentence,rep_init in zip(range(nrows),sentences,rep_initiators):
#     if rep_init== "repair_initiator":
#         sentences_window[sentence] = sentences[k-window_width:k+window_width+1]
# #         repair.append(list(sentences[k-window_width:k+window_width+1]))

# # rep=pd.DataFrame(repair).transpose()
# rep=pd.DataFrame(sentences_window)#.transpose()
# repair_tok = list(read_corpus(rep.iloc[:,0], tokens_only=True))

####################################################################                                                   

# for window in repair:
#     for ut in window:
#         print(type(ut))
#         tok = list(read_corpus(ut, tokens_only=True))
#         print(tok)
#         inferred_vector = model.infer_vector(tok)
#         sims = model.dv.most_similar([inferred_vector]) #sims[0] is most similar 

# for key,value in enumerate(sentences_window.keys()):     
#     for sentence_token in read_corpus(list(value), tokens_only=True):
#         inferred_vector = model.infer_vector(sentence_token)
#         sims = model.dv.most_similar([inferred_vector]) #sims[0] is most similar     
      
# list_inferred_in_window = []

# for key in enumerate(sentences_window.keys()): 
#     for tokenized_sent in sentences_window.values():
#         tokenize = read_corpus(tokenized_sent, tokens_only=True)
#         inferred_vector = model.infer_vector(tokenized_sents.words)
#         print(inferred_vector)
#         list_inferred_in_window.append(inferred_vector)
# #         sims = model.dv.most_similar([inferred_vector]) #sims[0] is most similar  



In [ ]:
#what i want is the cosine similarity of all 5 sentences within the window against each other
#compare vector against vector

from sklearn.metrics import pairwise


# for key in sentences_window.keys():
#     for e,window in enumerate(sentences_window.values()):
#         counter = e+1
#         print(window)
#         for ut in window:
#             print(ut)
#             while counter <= len(sentences_window.values()):
#                 print(window[counter])
#                 pair_sim = pairwise.cosine_similarity(window[e].reshape(1, -1), window[counter].reshape(1, -1))
#                 print(pair_sim)
#                 counter +=1
#             print(counter)
#             e+=1

            
# inf_vecs = []
# for ut in repair_tok:
#     inferred_vector = model.infer_vector(ut)
#     inf_vecs.append(inferred_vector)


inf_vecs = []
[inf_vecs.append(model.infer_vector(ut)) for ut in repair_tok]


# for e,vec in enumerate(inf_vecs):
#     while e < len(inf_vecs):
#         counter = e+1
#         while counter < len(inf_vecs):
#             pair_sim = pairwise.cosine_similarity(inf_vecs[e].reshape(1, -1), inf_vecs[counter].reshape(1, -1))
#             print(e, counter, pair_sim)
#             counter +=1
#         e+=1
#         if (counter == len(inf_vecs)) & (e == len(inf_vecs)-1):
#             break
    

threshold = 0.2
paraphrasing = []

for e in range(len(inf_vecs)):
    counter = e+1
    while counter < len(inf_vecs):
        pair_sim = pairwise.cosine_similarity(inf_vecs[e].reshape(1, -1), inf_vecs[counter].reshape(1, -1))
        print(e, counter, pair_sim)
        if pair_sim >= threshold:
            print('YES', str(e), str(counter), str(pair_sim))
            paraphrasing.append((e,counter,pair_sim))
        counter +=1

paraphrasing

# pairwise.cosine_similarity(list_inferred_in_window[0].reshape(1, -1), list_inferred_in_window[1].reshape(1, -1))

In [ ]:
#this works, but the dictionary solution is more elegant to create the windows
# N = 2
# # End slice ensures same default min_periods behavior to `.rolling`
# df1 = pd.concat([df['index'].shift(i).rename(i) for i in range(N)], axis=1).iloc[N-1:]
# df1

# testing stuff

In [240]:
# doesn't make a difference to add \ before the quote. ex: "I\'m fine."
# correction = []
# for e, text in enumerate(df['new_text']):
#     correction.append(text.replace("'", r"\'"))

# len(correction)
# df['new_text2'] = correction

In [258]:
paraphr

{'dlg-c36bfbe7-d2e5-43dd-9e29-72361ea84d3b': [(0,
   'What is the cheapest flight?',
   1,
   'The cheapest flight is with American airlines at $559.',
   array([[0.72890043]], dtype=float32)),
  (0,
   'What is the cheapest flight?',
   4,
   'Then Swiss airlines at $687.',
   array([[0.32189715]], dtype=float32))],
 'dlg-8fe9ee31-81df-4047-9085-fac75fa855e5': [(0,
   'Anything else?',
   2,
   'What was the last thing you said?',
   array([[0.31299445]], dtype=float32)),
  (0,
   'Anything else?',
   4,
   'Okay, anything else?',
   array([[0.7684341]], dtype=float32)),
  (1,
   'I want the hotel with a restaurant and a fitness-center.',
   3,
   'The hotel must have a fitness-center.',
   array([[0.4102459]], dtype=float32))],
 'dlg-a0d0b8e9-6b6f-4b8e-ba2c-35f9f3b34431': [(0,
   'Golden City Restaurant, I think I heard some good things.',
   4,
   "I think I\\'ll check out this place.",
   array([[0.39901876]], dtype=float32)),
  (1,
   'Now, does that have outdoor seating?',
   2,
